In [14]:
import sys
import IPython
import numpy as np
import pandas as pd
import sklearn as sk

print ('Python version: %s.%s.%s' % sys.version_info[:3])
print ('IPython version:', IPython.__version__)
print ('numpy version:', np.__version__)
print ('pandas version:', pd.__version__)
print ('scikit-learn version:', sk.__version__)

Python version: 3.6.3
IPython version: 6.2.1
numpy version: 1.13.3
pandas version: 0.21.0
scikit-learn version: 0.19.1


In [17]:
#leyendo datos creados desde Rstudio
import feather
walmart = feather.read_dataframe('../data/transformed_data.feather')
walmart.head()

,VisitNumber,TripType,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,DepartmentGroup,numItems,num_purchased
0,10,8,friday,6414410235,11,dsd grocery,2008,food,3,33
1,10,8,friday,2800053970,11,"candy, tobacco, cookies",115,food,3,33
2,10,8,friday,7794800902,11,dsd grocery,7950,food,3,33
3,100,37,friday,4383,11,produce,3102,food,1,11
4,1000,9,friday,32878550911,11,infant consumable hardlines,2009,infant,1,11


In [18]:
data2 = pd.get_dummies(walmart, columns =['Weekday', 'DepartmentGroup'])
columns = ['Upc', 'FinelineNumber', 'ScanCount']#, 'VisitNumber']
data2.drop(columns, inplace=True, axis=1)
print(data2.loc[data2['VisitNumber'] == '10'])

  VisitNumber TripType    DepartmentDescription  numItems  num_purchased  \
0          10        8              dsd grocery         3             33   
1          10        8  candy, tobacco, cookies         3             33   
2          10        8              dsd grocery         3             33   

   Weekday_friday  Weekday_monday  Weekday_saturday  Weekday_sunday  \
0               1               0                 0               0   
1               1               0                 0               0   
2               1               0                 0               0   

   Weekday_thursday          ...            DepartmentGroup_cloth  \
0                 0          ...                                0   
1                 0          ...                                0   
2                 0          ...                                0   

   DepartmentGroup_media and gaming  DepartmentGroup_house  \
0                                 0                      0   
1        

In [127]:
#aux = data2.groupby(["VisitNumber"])
#print(aux.head(10))

aux = data2
temp = data2[['VisitNumber','TripType']].drop_duplicates(["VisitNumber", "TripType"])
#print(aux.columns.values)

#columns = ['TripType'] #,  'numItems',  'num_purchased' ]#, 'VisitNumber']
#aux.drop(columns, inplace=True, axis=1)

In [128]:
aux = aux.groupby(["VisitNumber"]).mean().reset_index()
aux = aux.dropna(axis=0, how='any')
print(aux.shape)
#print(aux.columns.values)
print(aux[aux['VisitNumber'] == '10'])

(95674, 25)
  VisitNumber  numItems  num_purchased  Weekday_friday  Weekday_monday  \
4          10         3             33             1.0             0.0   

   Weekday_saturday  Weekday_sunday  Weekday_thursday  Weekday_tuesday  \
4               0.0             0.0               0.0              0.0   

   Weekday_wednesday          ...            DepartmentGroup_cloth  \
4                0.0          ...                              0.0   

   DepartmentGroup_media and gaming  DepartmentGroup_house  \
4                               0.0                    0.0   

   DepartmentGroup_girls wear, 4-6x  and 7-14  DepartmentGroup_home  \
4                                         0.0                   0.0   

   DepartmentGroup_garden  DepartmentGroup_infant  DepartmentGroup_null  \
4                     0.0                     0.0                   0.0   

   DepartmentGroup_office  DepartmentGroup_games  
4                     0.0                    0.0  

[1 rows x 25 columns]


In [129]:
df = aux.join(temp.set_index('VisitNumber'), on='VisitNumber')
print(df)

      VisitNumber  numItems  num_purchased  Weekday_friday  Weekday_monday  \
0               5         1             10             1.0             0.0   
1               7         2             22             1.0             0.0   
2               8        23            260             1.0             0.0   
3               9         3             33             1.0             0.0   
4              10         3             33             1.0             0.0   
5              11         4             44             1.0             0.0   
6              12         7             77             1.0             0.0   
7              15         8             89             1.0             0.0   
8              17         4             44             1.0             0.0   
9              19         9             99             1.0             0.0   
10             20         2             23             1.0             0.0   
11             23         2             22             1.0      

In [130]:
## Usando OneHotEncoder
from sklearn.preprocessing import LabelEncoder
#from sklearn.preprocessing import OneHotEncoder

#print(iris_df.head())
X = df.iloc[:, df.columns != 'TripType']
y = df.iloc[:, df.columns=="TripType"]

label_encoder = LabelEncoder()  ## Para convertir a enteros
#one_hot_encoder = OneHotEncoder()

## Convertirmos a enteros, i.e. setosa -> 0, etc
y = label_encoder.fit_transform(y)
print(y[0:10],)
print(y.shape)

[37 16 11 35 35 21 28  6 33 29]
(95674,)


/Users/timgivois/personal/venv3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [131]:
print(X.head(4))

  VisitNumber  numItems  num_purchased  Weekday_friday  Weekday_monday  \
0           5         1             10             1.0             0.0   
1           7         2             22             1.0             0.0   
2           8        23            260             1.0             0.0   
3           9         3             33             1.0             0.0   

   Weekday_saturday  Weekday_sunday  Weekday_thursday  Weekday_tuesday  \
0               0.0             0.0               0.0              0.0   
1               0.0             0.0               0.0              0.0   
2               0.0             0.0               0.0              0.0   
3               0.0             0.0               0.0              0.0   

   Weekday_wednesday          ...            DepartmentGroup_cloth  \
0                0.0          ...                              0.0   
1                0.0          ...                              0.5   
2                0.0          ...               

In [132]:
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split 
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2)

# Magic Loop

[tutorial](http://www.codiply.com/blog/hyperparameter-grid-search-across-multiple-models-in-scikit-learn/)

*First  dictionary:* models to be scored <br>
*Second dictionary:* parameters for each model <br>
*Fit:* returns a paremeter grid search with cross validation for each model and for the given data <br>
*Score_summary:* returns a data_frame with a summary of the scores <br>

In [133]:
from sklearn.grid_search import GridSearchCV

class EstimatorSelectionHelper:
    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}
    
    def fit(self, X, y, cv=3, n_jobs=1, verbose=1, scoring=None, refit=True):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, 
                              params, 
                              cv=cv, 
                              n_jobs=n_jobs, 
                              verbose=verbose, 
                              scoring=scoring, 
                              refit=refit)
            gs.fit(X,y)
            self.grid_searches[key] = gs    
    
    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'min_score': min(scores),
                 'max_score': max(scores),
                 'mean_score': np.mean(scores),
                 'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})
            #return pd.Series(dict(params.items() + d.items()))
                      
        rows = [row(k, gsc.cv_validation_scores, gsc.parameters) 
                for k in self.keys
                for gsc in self.grid_searches[k].grid_scores_]
        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)
        
        columns = ['estimator', 'min_score', 'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]
        
        return df[columns]


In [141]:
from sklearn.ensemble import (ExtraTreesClassifier, RandomForestClassifier, 
                              AdaBoostClassifier, GradientBoostingClassifier)
from sklearn.svm import SVC
from sklearn.linear_model import  LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

models1 = { 
    'ExtraTreesClassifier': ExtraTreesClassifier(),
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    #'LogisticRegression' : LogisticRegression(),
    'KNeighborsClassifier' : KNeighborsClassifier(),
    #'NaiveBayes': MultinomialNB(),
    'SVC': SVC()
}

params1 = { 
    'ExtraTreesClassifier': { 'n_estimators': [16, 32] },
    'RandomForestClassifier': { 'n_estimators': [15, 30, 50], 'max_depth': [10,100], 'max_features':['sqrt', 'log2'], 'min_samples_split': [5,10] },
    'AdaBoostClassifier':  { 'n_estimators': [16, 32] },
    'GradientBoostingClassifier': { 'n_estimators': [16, 32], 'learning_rate': [0.8, 1.0] },
    #'LogisticRegression' : { 'C' : [1, 1e3, 1e5] },
    'KNeighborsClassifier' : { 'n_neighbors' : [3,5] },
    #'NaiveBayes' : { 'alpha' : [0.1, 0.001, 0.0001] },
    'SVC': [
        {'kernel': ['linear'], 'C': [1, 10, 100]},
        {'kernel': ['rbf'], 'C': [1, 10, 100], 'gamma': [0.001, 0.0001]},
    ]
}


In [ ]:
helper1 = EstimatorSelectionHelper(models1, params1)
%time helper1.fit(X_train, y_train, scoring='accuracy', n_jobs=-1)

Running GridSearchCV for ExtraTreesClassifier.
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   10.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   10.0s finished


Running GridSearchCV for RandomForestClassifier.
Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   33.1s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:  1.2min finished


Running GridSearchCV for AdaBoostClassifier.
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    7.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    7.0s finished


Running GridSearchCV for GradientBoostingClassifier.
Fitting 3 folds for each of 4 candidates, totalling 12 fits


In [138]:
helper1.score_summary(sort_by='min_score')

,estimator,min_score,mean_score,max_score,std_score,max_depth,max_features,min_samples_split,n_estimators
23,RandomForestClassifier,0.515441,0.515842,0.516059,0.000283446,100,log2,10,50
17,RandomForestClassifier,0.51454,0.516495,0.518295,0.0015369,100,sqrt,10,50
0,RandomForestClassifier,0.513667,0.515123,0.517283,0.00155814,10,sqrt,5,15
22,RandomForestClassifier,0.51355,0.513712,0.514026,0.000222158,100,log2,10,30
16,RandomForestClassifier,0.512616,0.513935,0.515706,0.00130186,100,sqrt,10,30
2,RandomForestClassifier,0.512106,0.513451,0.515285,0.00134288,10,sqrt,5,50
5,RandomForestClassifier,0.511992,0.51268,0.513517,0.000631208,10,sqrt,10,50
4,RandomForestClassifier,0.510817,0.513086,0.517354,0.0030197,10,sqrt,10,30
15,RandomForestClassifier,0.509089,0.509505,0.509863,0.000318451,100,sqrt,10,15
21,RandomForestClassifier,0.508426,0.508813,0.509275,0.00035063,100,log2,10,15



# Evaluating model

In [37]:
model = RandomForestClassifier(n_estimators=50, max_features='sqrt', max_depth=100, min_samples_split=5) #, gamma=0.001)

%time model.fit(X_train, y_train)
print(model.score(X_train, y_train))
y_pred = model.predict(X_test)
print (confusion_matrix(y_test, y_pred))
print (classification_report(y_test, y_pred))

CPU times: user 49.5 s, sys: 1.43 s, total: 51 s
Wall time: 51.4 s
0.991157998852
[[ 413    0    0 ...,    0    4    0]
 [   0   10    0 ...,    0    0    0]
 [   0    0 1334 ...,   11    4    1]
 ..., 
 [   0    0    6 ..., 3672  163   47]
 [   2    0    2 ...,  181 2785   59]
 [   0    0    0 ...,  132   87 3112]]
             precision    recall  f1-score   support

          0       0.98      0.97      0.98       424
          1       1.00      1.00      1.00        10
          2       0.97      0.96      0.97      1386
          3       0.96      0.93      0.95       556
          4       0.84      0.79      0.82       211
          5       0.94      0.93      0.93       619
          6       0.98      0.97      0.97       819
          7       0.89      0.85      0.87       745
          8       0.90      0.83      0.86        69
          9       0.98      0.96      0.97      3610
         10       0.98      0.99      0.99      5431
         11       0.95      0.89      0.92   

In [56]:
walmart_test = feather.read_dataframe('../data/test_transformed_data.feather')
walmart_test.head()


,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,DepartmentGroup,numItems,num_purchased
0,1,friday,72503389714,1,shoes,3002,cloth,4,4
1,1,friday,1707710732,1,dairy,1526,food,4,4
2,1,friday,89470001026,1,dairy,1431,food,4,4
3,1,friday,88491211470,1,grocery dry goods,3555,food,4,4
4,10000,saturday,87458604264,1,impulse merchandise,8023,other departments,1,1


In [67]:
data3 = pd.get_dummies(walmart_test, columns =['Weekday', 'DepartmentGroup'])
columns = ['Upc', 'FinelineNumber', 'ScanCount', 'DepartmentDescription']#, 'VisitNumber']
data3.drop(columns, inplace=True, axis=1)


In [98]:
aux = data3
temp = data3[['VisitNumber']]
aux = aux.groupby(["VisitNumber"]).mean().reset_index()
aux = aux.dropna(axis=0, how='any')
df_test_final = aux.join(temp.set_index('VisitNumber'), on='VisitNumber')
df_test_final.head()

,VisitNumber,numItems,num_purchased,Weekday_friday,Weekday_monday,Weekday_saturday,Weekday_sunday,Weekday_thursday,Weekday_tuesday,Weekday_wednesday,...,DepartmentGroup_cloth,DepartmentGroup_media and gaming,DepartmentGroup_house,"DepartmentGroup_girls wear, 4-6x and 7-14",DepartmentGroup_home,DepartmentGroup_garden,DepartmentGroup_infant,DepartmentGroup_null,DepartmentGroup_office,DepartmentGroup_games
0,1,4,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,1,4,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,1,4,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,1,4,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,4,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
y_test_final = model.predict(data3)
y_test_final

array([37, 37, 37, ..., 37, 37, 37])

In [95]:
y_final = label_encoder.inverse_transform(y_test_final)


array(['999', '999', '999', ..., '999', '999', '999'], dtype=object)